<a href="https://colab.research.google.com/github/parus-cristatus/tolokapizza/blob/main/tolokakit/AssignmentsObserver_with_log.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install toloka-kit
!pip install nest-asyncio

In [3]:
import logging
import sys
import toloka.client as toloka
from datetime import timedelta
from toloka.streaming import Pipeline, AssignmentsObserver

In [ ]:
target = 'SANDBOX' #PRODUCTION
toloka_client = toloka.TolokaClient(input("Enter your token:"), target)
print(toloka_client.get_requester())

In [5]:
pool = toloka_client.get_pool(pool_id='1315173')

In [6]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(filename='log.log',
                    filemode='a',
                    format='%(asctime)s %(message)s', 
                    datefmt='%d.%m.%Y %H:%M:%S',
                    level=logging.INFO
                    )

In [7]:
def handle_accepted(events):
    for event in events:
        logging.info(f"{event}")

In [8]:
observer = AssignmentsObserver(toloka_client, pool_id=pool.id)
observer.on_accepted(handle_accepted)

<function __main__.handle_accepted(events)>

In [ ]:
pipeline = Pipeline(period=timedelta(seconds=10))
pipeline.register(observer)

pool = toloka_client.open_pool(pool.id)
print(f'pool - {pool.status}')

In [ ]:
IN_COLAB = 'google.colab' in sys.modules
IN_COLAB

In [12]:
if IN_COLAB:
    import nest_asyncio, asyncio
    nest_asyncio.apply()
    asyncio.get_event_loop().run_until_complete(pipeline.run())